<a href="https://colab.research.google.com/github/Gongsoo/Python3/blob/main/MNIST_CNN_STUDY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import torch
import torch.nn as nn

inputs = torch.Tensor(1,1,28,28)

conv1 = nn.Conv2d(1,32,3,padding=1)
conv2 = nn.Conv2d(32,64,kernel_size = 3,padding=1)
pool = nn.MaxPool2d(2)
fc = nn.Linear(3136,10)
out = conv1(inputs)
out = pool(out)
out = conv2(out)
out = pool(out)

out = out.view(out.size(0),-1)
out = fc(out)

In [16]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

torch.manual_seed(777)

if device == 'cuda' :
  torch.cuda.manual_seed_all(777)

cuda


In [17]:
lr = 0.01
epochs = 15
batch_size = 100

mnist_train = dsets.MNIST(root = 'MNIST_data', train = True, transform = transforms.ToTensor(),download=True)
mnist_test = dsets.MNIST(root = 'MNIST_data', train = False, transform = transforms.ToTensor(),download=True)
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,batch_size = batch_size,shuffle = True,drop_last = True)



In [18]:
class CNN(torch.nn.Module) :
  def __init__(self) :
    super(CNN, self).__init__()
    self.keep_prob = 0.5
    self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(1,32,kernel_size = 3, stride = 1, padding = 1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.layer2 = torch.nn.Sequential(
        torch.nn.Conv2d(32,64,kernel_size = 3, stride = 1, padding = 1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.layer3 = torch.nn.Sequential(
        torch.nn.Conv2d(64,128,kernel_size = 3, stride = 1, padding = 1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride=2,padding=1)
    )
    self.fc1 = torch.nn.Linear(4*4*128,625,bias=True)
    torch.nn.init.xavier_uniform_(self.fc1.weight)
    self.layer4 = torch.nn.Sequential(
        self.fc1,
        torch.nn.ReLU(),
        torch.nn.Dropout(p=1-self.keep_prob)
    )
    self.fc2 = torch.nn.Linear(625,10,bias = True)
    torch.nn.init.xavier_uniform_(self.fc2.weight)

  def forward(self, x) :
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)
    out = out.view(out.size(0),-1)
    out = self.layer4(out)
    out = self.fc2(out)
    return out

In [19]:
model = CNN().to(device)

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = lr)

In [20]:
for epoch in range(epochs) :
  avg_cost = 0

  for X,Y in data_loader :
    X = X.to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis,Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost/len(data_loader)
  
  print('[Epoch: {:>4} cost = {:>.9}'.format(epoch+1,avg_cost))

[Epoch:    1 cost = 0.393967003
[Epoch:    2 cost = 0.202635407
[Epoch:    3 cost = 0.17179437
[Epoch:    4 cost = 0.158677757
[Epoch:    5 cost = 0.14911063
[Epoch:    6 cost = 0.144889668
[Epoch:    7 cost = 0.142894909
[Epoch:    8 cost = 0.140646517
[Epoch:    9 cost = 0.13554661
[Epoch:   10 cost = 0.1383733
[Epoch:   11 cost = 0.14045015
[Epoch:   12 cost = 0.140136912
[Epoch:   13 cost = 0.14092733
[Epoch:   14 cost = 0.130244896
[Epoch:   15 cost = 0.135087296


In [22]:
# 학습을 진행하지 않을 것이므로 torch.no_grad()
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.7865999937057495
